# Application of MLFlow Model Registry

In this notebook, we are going to apply and register our models with MLFlow Model Registry. In order to achieve it, we are going to reuse the example 1 but, this time, there is only two notebooks. One is the Decision Tree model and the other is a new model: XGBoost.

### Running the Decision Tree Model

In [3]:
%run "./Models/Decision Tree"

### Registering the model
Now that our model is ready, we are going to register it. To do this, we are going to use the Tracking UI.

#### Creating a new Registered Model
1. Got into the MLFlow Experiment, click on the model
2. Now, go to the artefacts part and select the `Decision Tree_Model` folder
3. Click on the `Register Model` button
![Image of creating a new registered model](https://raw.githubusercontent.com/StevanStanovic/mlflow/master/4%20-%20MLFlow%20Model%20Registry%20et%20d%C3%A9ploiement/Images/Creating_new_registered_model.png)

4. Create a new MLFlow Registered Model and give it a name (for example: MLFlow Registered Model test)
5. After creating it, click on `Models` button in the task bar and select your new registered model

### Presenting the interface

Normally, your registred model looks like :

![Image of registered model interface](https://raw.githubusercontent.com/StevanStanovic/mlflow/master/4%20-%20MLFlow%20Model%20Registry%20et%20d%C3%A9ploiement/Images/Registered_model_interface.png)

You can add a description for your registered model, select a version and compare them.

Select the version 1 and you will see:
1. All informations about this version
2. A Stage button, it is useful because you can request transition and make some transtions. There is three transitions: Staging, Production and Archived.
3. A Source Run, this link redirects you to your experiment and therefore your code
4. A description
5. And all informations about the management of your model

Now, we are going to add a description and request a Production transition.
1. Go to the button on the right of `Description`, add a description (for example: Getting started with MLFlow Registry Model) and save it
2. The description is now visible
3. Click on the `Stage` button and then on the `Request transition to -> Production` button
4. Add a comment (for example: It is my first registered model) and click on `OK`
5. In the Pending Requests part, your request has just appeared and this new activity has registered also in the Activities part
![Image of the request appearition](https://raw.githubusercontent.com/StevanStanovic/mlflow/master/4%20-%20MLFlow%20Model%20Registry%20et%20d%C3%A9ploiement/Images/The_request_appearition.png)
6. Click on `Approve`, comment it (for exemple: OK for me) and click on `Confirm`

In the end, your registred model looks like :

![Image of registered model interface 2](https://raw.githubusercontent.com/StevanStanovic/mlflow/master/4%20-%20MLFlow%20Model%20Registry%20et%20d%C3%A9ploiement/Images/Registered_model_interface_2.png)

#### Now that we know how to register a model and make transitions, we are going to:
1. Load the registered model in two different ways
2. Run the XGBoost script and compare it with the Decision Tree model
3. Manage some transition tasks with script lines

### Loading the registered model
We are going to do that with the `mlflow.pyfunc` module as in example 3 but, this time, we will give a version and/or production URI.

In [9]:
import mlflow.pyfunc

register_name = "MLFlow Registered Model test"

#FIRST WAY: using the version 1 URI
version_uri = "models:/{}/1".format(register_name)
version1 = mlflow.pyfunc.load_model(version_uri)

#SECOND WAY: using the production URI
production_uri = "models:/{}/production".format(register_name)
production = mlflow.pyfunc.load_model(production_uri)

**Remark: If we have some instances, we can predict them with the method `predict`.**

### Running the XGBoost script and compare it with the Decision Tree model

In [12]:
%run "./Models/XGBoost"

####Accuracy comparison between two models

![Image of comparison of two models](https://raw.githubusercontent.com/StevanStanovic/mlflow/master/4%20-%20MLFlow%20Model%20Registry%20et%20d%C3%A9ploiement/Images/Comparison_two_models.PNG)

We see clearly that XGBoost model is better than Decision Tree model !
So, we are going to create a new version for our registered model and see how to manage this version and what we can do with a script.

**Remark: If you want to know more about implementing models management, the [following link](https://www.mlflow.org/docs/latest/model-registry.html#api-workflow) explains it.**

### Adding a new version in the register model

In [16]:
# Informations about our XGBoost model
run_ID = ###Set_your_run_ID_using_a_string
artifact_name = "XGBoost_Model"

# Creating a new version
mlflow.register_model(
  "runs:/{}/{}".format(run_ID,artifact_name),
  register_name
)

model_version = 2

### Adding or updating a description for the new model version

In [18]:
from mlflow.tracking.client import MlflowClient

client = MlflowClient()
client.update_model_version(
  name=register_name,
  version=model_version,
  description="This model is better than the first model version"
)

### Renaming a registered model

In [20]:
"""
new_register_name = "New name"

client.rename_registered_model(
    name=register_name,
    new_name=new_register_name
)
"""

### Transitioning  a model version to Staging state

In [22]:
client.transition_model_version_stage(
  name=register_name,
  version=model_version,
  stage="Staging"
)

### Transitioning  a model version to Production state

In [24]:
client.transition_model_version_stage(
  name=register_name,
  version=model_version,
  stage="Production"
)

### Transitioning  a model version to Archived state

In [26]:
"""
client.transition_model_version_stage(
  name=register_name,
  version=model_version,
  stage="Archived"
)
"""

### Deleting the first model version

In [28]:
client.delete_model_version(
  name=register_name,
  version=1
)

### Deleting a register model

In [30]:
"""
client.delete_registered_model(
  name=register_name
)
"""

# Deployement of an algorithm on Azure Machine Learning (AML)

### Installing azureml and azureml.core modules on the cluster

1. Go to `Clusters` in the task bar and select your cluster
2. Click on `Libraries`, then on `Install New` and finally on `PyPl`
3. Enter the name module that you want to install (in our case mlflow or mlflow.core)
4. Click on `Install` and wait until the end of module installation

### Importing Python modules

In [36]:
import mlflow.azureml
from azureml.core import Workspace
from azureml.core.webservice import AciWebservice, Webservice

### Creating an Azure ML Workspace

In [38]:
workspace_name = "<Name of your Azure ML workspace>"
subscription_id = "<Your Azure subscription ID>"
resource_group = "<Name of the Azure resource group in which to create Azure ML resources>"
location = "<Name of the Azure location (region) in which to create Azure ML resources>"
azure_workspace = Workspace.create(name=workspace_name,
                                   subscription_id=subscription_id,
                                   resource_group=resource_group,
                                   location=location,
                                   create_resource_group=True,
                                   exist_ok=True)

### Building an Azure ML Container Image for an MLflow model

In [40]:
azure_image, azure_model = mlflow.azureml.build_image(model_uri="models:/{}/production".format(register_name),
                                                      workspace=azure_workspace,
                                                      synchronous=True)
# If your image build failed, you can access build logs at the following URI:
print("Access the following URI for build logs: {}".format(azure_image.image_build_log_uri))

### Deploying the image to Azure Container Instances (ACI) for real-time serving

In [42]:
webservice_deployment_config = AciWebservice.deploy_configuration()
webservice = Webservice.deploy_from_image(
  image=azure_image, 
  workspace=azure_workspace, 
  name="test")
webservice.wait_for_deployment()

If you check your Azure account and go to your ressource group, you will see that four bricks have been created

![Images of creation of four bricks in AML](https://raw.githubusercontent.com/StevanStanovic/mlflow/master/4%20-%20MLFlow%20Model%20Registry%20et%20d%C3%A9ploiement/Images/Creation_in_AML.png)

**Remark: If you want to know more about AML deployement using MLFlow API, click on this [link](https://www.mlflow.org/docs/latest/python_api/mlflow.azureml.html).**